# Import Dependensi

Package

In [1]:
import pandas as pd
import numpy as np

Modul Perhitungan ET

In [2]:
from modul.gravimetri import gravimetri_central_diff as gravimetri
from modul.bc_model import bc_model
from modul.hs_model import hs_model
from modul.pm_model import pm_model

Variabel

In [3]:
from variabel import variabel as var

# Import Dataset

In [4]:
# Import file excel
dataset_primer = 'data/dataset_primer_3_processed.xlsx'
dataset_sekunder = 'data/dataset_sekunder_2.xlsx'

# Membaca semua sheet dari file Excel
data_primer = pd.ExcelFile(dataset_primer)
data_sekunder = pd.ExcelFile(dataset_sekunder)

# Fungsi-Fungsi

In [5]:
# Menghitung luas area di dalam variabel
def hitung_luas_media(sisi):
        cm_to_mm = sisi[0] * 10
        luas = (cm_to_mm ** 2)
        return luas              # Hasil dalam satuan mm

In [6]:
# Menghitung tanggal pengambilan data
def hitung_tanggal_data(day):
    day = int(day.split()[1])

    tahun = '2024'

    if day > 10:
        bulan = '11'
    else:
        bulan = '10'
    
    def hitung_tanggal(input_day):
        # Jika x antara 1 dan 31, y adalah x + 20, tetapi melingkar kembali ke awal setelah 31
        return (input_day + 21) % 31 if (input_day + 21) % 31 != 0 else 31

    hari = hitung_tanggal(day)
    hari = f"{hari:02d}"

    tanggal = (f"{tahun}-{bulan}-{hari}")

    return tanggal

# ET Model Gravimetri/Penimbangan

Perhitungan Gravimetri

In [7]:
hasil_gravimetri = []

for sheet_name in data_primer.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_primer = data_primer.parse(sheet_name)
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    for kode in var.sisi_media_tanaman.keys():
        kode_berat = f'berat_{kode}'
        kode_irigasi = f'supply_air_{kode}'
        
        # Buat array untuk variabel yang akan digunakan
        waktu = df_data_primer['waktu'].array
        berat = df_data_primer[kode_berat].array                            # Satuan kg
        irigasi_liter = df_data_primer[kode_irigasi].fillna(0).array        # Satuan liter
        curah_hujan = df_data_sekunder['curah_hujan'].array                 # Satuan mm

        # Hitung luas alas media tanaman
        luas_media = hitung_luas_media(var.sisi_media_tanaman[kode])    # Satuan mm^2

        # Ubah irigasi ke satuan mm
        irigasi_mm = (irigasi_liter * 1e6) / (luas_media)               # Satuan mm

        # Ubah berat ke satuan volume mm^3
        volume = berat * 1e6                                            # Satuan mm^3

        # Hitung total supply air
        supply_air = irigasi_mm + curah_hujan                            # Satuan mm

        perubahan_berat, perubahan_et, et_total = gravimetri(berat=volume,
                                                            waktu=waktu,
                                                            luas=luas_media,
                                                            supply_air=supply_air
                                                            )

        # Simpan hasil ke list
        hasil_gravimetri.append({
            "Num Day": sheet_name,
            "Kode": kode,
            "ET Total": et_total
        })

# Buat DataFrame dari hasil
df_hasil_gravimetri = pd.DataFrame(hasil_gravimetri)

# Pivot tabel dengan "Day" sebagai index dan "Kode" sebagai kolom
df_hasil_gravimetri = df_hasil_gravimetri.pivot(index="Num Day", columns="Kode", values="ET Total")


In [8]:
df_hasil_gravimetri.to_csv('hasil/ET Aktual Gravimetri.csv', index=True)
df_hasil_gravimetri.head(10)

Kode,A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,...,D3,D4,E1,E2,E3,E4,F1,F2,F3,F4
Num Day,,,,,,,,,,,,,,,,,,,,,
day 05,8.4480,5.0304,8.7360,12.5952,2.173333,4.866667,3.826667,5.533333,10.755918,8.551837,...,6.37500,9.46500,1.108148,2.951111,0.622222,0.628148,2.3376,0.8592,1.1616,2.8896
day 06,1.2084,4.9140,3.9156,6.2004,6.124667,4.524667,4.258000,3.658000,4.935551,2.447388,...,5.21550,1.96050,1.855037,3.780963,1.428370,3.863926,0.8940,1.5948,1.1580,2.6268
day 07,8.1024,7.7664,5.6256,6.6432,5.853333,7.273333,6.346667,5.546667,5.030204,3.075918,...,4.05000,1.96125,4.598519,6.334815,4.968889,4.530370,0.0552,1.0680,2.2296,3.1896
day 08,7.0656,4.2624,6.2208,6.8736,6.653333,8.853333,5.866667,5.120000,5.201633,4.986122,...,4.69500,9.18750,2.186667,4.640000,3.591111,4.977778,0.2496,0.5088,0.8640,0.7200
day 09,5.7612,5.3772,5.6268,6.1836,8.847333,8.047333,7.107333,5.267333,7.662980,6.443388,...,3.20775,5.19525,2.728074,4.695481,3.854000,2.891037,0.9204,0.9012,1.3356,1.1076
day 10,6.6564,5.4276,6.9060,7.1172,9.343333,9.116667,8.303333,7.023333,6.398571,4.997755,...,4.98750,5.80500,3.698889,5.322593,3.278148,2.756667,0.5220,2.6484,4.5876,5.8692
day 12,5.8728,5.7192,4.9128,4.0488,5.953333,5.486667,5.940000,7.806667,4.935918,5.180816,...,4.36500,19.55250,5.474815,5.255556,5.243704,6.837778,7.8456,8.0616,7.1496,7.2264
day 13,4.2324,3.5412,3.5988,3.0420,2.420667,1.714000,9.007333,3.780667,2.548286,16.997265,...,3.53400,3.54900,2.380667,4.679926,5.266593,4.745111,5.3892,6.5892,5.0340,5.6676
day 14,7.9008,7.9392,7.2096,7.0560,3.261333,9.725333,4.741333,5.261333,4.289633,4.064327,...,4.27050,6.91050,3.834667,4.693926,4.776889,3.905778,7.9440,6.3408,5.6256,5.5968


# ET Model Blaney-Criddle

In [9]:
hasil_bc_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)

    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")
    
    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    # print(f"Suhu udara {sheet_name}: {suhu_udara}")

    # Menghitung rata-rata suhu udara
    mean_suhu_udara = np.mean(suhu_udara)
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Blaney-Criddle
    et_model_bc = bc_model(var.persentase_hari_terang, mean_suhu_udara)
    # print(f"ET Blaney-Criddle: {et_model_bc}\n")

    # Simpan hasil ke list
    hasil_bc_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Mean Suhu Udara": mean_suhu_udara,
        "ET Total": et_model_bc
    })

# Buat DataFrame dari hasil
df_hasil_bc = pd.DataFrame(hasil_bc_model)

In [10]:
df_hasil_bc.to_csv('hasil/Estimasi ET Blaney-Criddle.csv', index=False)
df_hasil_bc.head()

,Num Day,Tanggal,Mean Suhu Udara,ET Total
0,day 05,2024-10-26,30.90750,6.498712
1,day 06,2024-10-27,31.77875,6.606921
2,day 07,2024-10-28,32.63000,6.712646
3,day 08,2024-10-29,32.22250,6.662035
4,day 09,2024-10-30,31.17125,6.531469


# ET Model Hargreaves-Samani

In [11]:
hasil_hs_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Hargreaves-Samani
    radiasi_ekstrateresterial, et_model_hs, et_model_hs_modification = hs_model(t_max = max_suhu_udara, 
                                                                                t_min = min_suhu_udara, 
                                                                                t_avg = mean_suhu_udara, 
                                                                                latitude = var.latitude, 
                                                                                input_date = tanggal_data)
    # print(f"ET Hargreaves-Samani: {et_model_hs}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_hs_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": et_model_hs,
    })

# Buat DataFrame dari hasil
df_hasil_hs = pd.DataFrame(hasil_hs_model)

In [ ]:
df_hasil_hs.to_csv('hasil/Estimasi ET Hargreaves-Samani.csv', index=False)
df_hasil_hs.head()

,Num Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Radiasi Extrateresterial,ET Total
0,day 05,2024-10-26,33.07,25.59,29.330,15.792283,5.881888
1,day 06,2024-10-27,34.18,25.99,30.085,15.798661,6.179544
2,day 07,2024-10-28,35.19,26.08,30.635,15.804507,6.514070
3,day 08,2024-10-29,34.46,26.55,30.505,15.809836,6.140098
4,day 09,2024-10-30,32.96,26.74,29.850,15.814667,5.522609


# ET Model Penman-Monteith

In [13]:
hasil_pm_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    kecepatan_angin = np.array(df_data_sekunder['kecepatan_angin'])
    kelembapan_relatif = np.array(df_data_sekunder['kelembapan_relatif'])
    
    # Menghitung rata-rata kecepatan angin dan kelembapan relatif
    mean_kecepatan_angin = np.mean(kecepatan_angin)
    # print(f"Rata-rata kecepatan angin: {mean_kecepatan_angin}")
    mean_kelembapan_relatif = np.mean(kelembapan_relatif)
    # print(f"Rata-rata kelembapan relatif: {mean_kelembapan_relatif}")

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")
    
    # Menghitung ET dengan model Penman-monteith
    radiasi_ekstrateresterial, et_model_pm, total_et_model_pm = pm_model(RH = mean_kelembapan_relatif, 
                                                                         u_2 = kecepatan_angin, 
                                                                         T_hr = suhu_udara, 
                                                                         T_avg = mean_suhu_udara, 
                                                                         T_max = max_suhu_udara, 
                                                                         T_min = min_suhu_udara, 
                                                                         altitude = var.altitude, 
                                                                         latitude = var.latitude, 
                                                                         input_date = tanggal_data)
    # print(f"ET Penman-monteith {sheet_name}: {total_et_model_pm}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_pm_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Mean Kecepatan Angin": mean_kecepatan_angin,
        "Mean Kelembapan Relatif": mean_kelembapan_relatif,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": total_et_model_pm
    })

# Buat DataFrame dari hasil
df_hasil_pm = pd.DataFrame(hasil_pm_model)

In [14]:
df_hasil_pm.to_csv('hasil/Estimasi ET Penman-Monteith.csv', index=False)
df_hasil_pm.head()

,Num Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Mean Kecepatan Angin,Mean Kelembapan Relatif,Radiasi Extrateresterial,ET Total
0,day 05,2024-10-26,33.07,25.59,29.330,3.80000,48.86750,15.792283,5.104162
1,day 06,2024-10-27,34.18,25.99,30.085,2.74875,45.59375,15.798661,4.876762
2,day 07,2024-10-28,35.19,26.08,30.635,2.78875,42.54750,15.804507,5.293068
3,day 08,2024-10-29,34.46,26.55,30.505,3.95125,44.36125,15.809836,5.737096
4,day 09,2024-10-30,32.96,26.74,29.850,4.02250,50.61750,15.814667,4.990299


# Data Anova

In [15]:
from variabel import variabel as var
print(var.tinggi_tanaman)

{'A1': (25,), 'A2': (40,), 'A3': (45,), 'A4': (60,), 'B1': (25,), 'B2': (40,), 'B3': (45,), 'B4': (60,), 'C1': (25,), 'C2': (40,), 'C3': (45,), 'C4': (60,), 'D1': (25,), 'D2': (40,), 'D3': (45,), 'D4': (60,), 'E1': (25,), 'E2': (40,), 'E3': (45,), 'E4': (60,), 'F1': (25,), 'F2': (40,), 'F3': (45,), 'F4': (60,)}


In [16]:
luas_permukaan_wadah = {
    key: (value[0] ** 2,) for key, value in var.sisi_media_tanaman.items()
}
print(luas_permukaan_wadah)

{'A1': (156.25,), 'A2': (156.25,), 'A3': (156.25,), 'A4': (156.25,), 'B1': (225.0,), 'B2': (225.0,), 'B3': (225.0,), 'B4': (225.0,), 'C1': (306.25,), 'C2': (306.25,), 'C3': (306.25,), 'C4': (306.25,), 'D1': (400.0,), 'D2': (400.0,), 'D3': (400.0,), 'D4': (400.0,), 'E1': (506.25,), 'E2': (506.25,), 'E3': (506.25,), 'E4': (506.25,), 'F1': (625.0,), 'F2': (625.0,), 'F3': (625.0,), 'F4': (625.0,)}


In [17]:
dataset_primer = 'data/dataset_primer_2_processed.xlsx'
data_primer = pd.ExcelFile(dataset_primer)

In [18]:
# Dictionary untuk menyimpan hasil
berat_intensitas = []

for sheet_name in data_primer.sheet_names:
    df = data_primer.parse(sheet_name)
    
    # Memfilter kolom yang mengandung "berat"
    berat_columns = [col for col in df.columns if "berat" in col]
    berat_df = df[berat_columns]
    
    # Mendapatkan nilai intensitas matahari
    intensitas_cahaya = df["intensitas_matahari"].mean()
    
    for col in berat_df.columns:
        kode = col.split('_')[1]  # Mendapatkan kode tanaman
        rata_rata_berat = berat_df[col].mean()
        
        # Menambahkan ke daftar
        berat_intensitas.append({
            "Kode": kode,
            "Day": sheet_name,
            "Berat_Tanaman": rata_rata_berat,
            "Intensitas_Matahari": intensitas_cahaya
        })

df_data_anova = pd.DataFrame(berat_intensitas)

# Menambahkan kolom tinggi tanaman dan luas permukaan wadah
df_data_anova["Tinggi_Tanaman"] = df_data_anova["Kode"].map(lambda x: var.tinggi_tanaman[x][0])
df_data_anova["Luas_Permukaan_Wadah"] = df_data_anova["Kode"].map(lambda x: luas_permukaan_wadah[x][0])

In [19]:
df_data_anova.to_csv('hasil/Data Anova.csv', index=False) 
df_data_anova

,Kode,Day,Berat_Tanaman,Intensitas_Matahari,Tinggi_Tanaman,Luas_Permukaan_Wadah
0,A1,day 05,2.181938,78100.00,25,156.25
1,A2,day 05,3.039562,78100.00,40,156.25
2,A3,day 05,3.915938,78100.00,45,156.25
3,A4,day 05,3.264563,78100.00,60,156.25
4,B1,day 05,3.590562,78100.00,25,225.00
...,...,...,...,...,...,...
235,E4,day 15,14.973562,65481.25,60,506.25
236,F1,day 15,21.806562,65481.25,25,625.00
237,F2,day 15,24.193188,65481.25,40,625.00
238,F3,day 15,23.506188,65481.25,45,625.00
